In [1]:
from ipywidgets import IntProgress, IntSlider, Dropdown, HTMLMath, interact, widgets, Image

# <span style="color: #FF0000">Détermination du chiffre représenté sur une image par comparaison avec les k plus proches voisins</span>

### Import des bibliothèques nécessaires

```python
from ipywidgets import  IntSlider, Dropdown, HTMLMath, interact, widgets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from random import randint

#Schémas interactifs sur jupyter
%matplotlib notebook
```

### Mise en mémoire des données du dataset MNIST et mise en forme des données

```python
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
images = X.reshape((-1, 28, 28))
```

### Séparation des éléments en ensemble d'entrainement et ensemble de test
Rééchantillonnage pour réduire la taille des données. On se limite à 6000 éléments ou moins pour attendre moins longtemps. Les résultats seraient bien entendu meilleurs avec plus de données. 


On sépare aussi ces éléments en un ensemble d'entrainement (80%) et un ensemble de test (20%)

In [9]:
valeurs = [('6000',0),('3000',1),('2000',2),('1500',3)]
erreurs = [0.0633, 0.0733, 0.0900, 0.1233]
kmin = [5,3,4,9]

indice = Dropdown(
    options=valeurs,
    value=0,
    description="Eléments:",
)

display(indice)

train = HTMLMath(
    value=r"Taille des données d'entrainement: {} éléments".format(int(valeurs[0][0])*8//10),
    placeholder=' ',
    description=' ',
)

test = HTMLMath(
    value=r"Taille des données de test: {} éléments".format(int(valeurs[0][0])*2//10),
    placeholder=' ',
    description=' ',
)

titreerreur = HTMLMath(
    value=r"<b>Erreur pour différentes valeurs de k avec un échantillon de {} éléments</b>".format(int(valeurs[0][0])),
    placeholder=' ',
    description=' ',
)

valeurk = HTMLMath(
    value=r"<b>La meilleure valeur de k est {} correspondant à {:.2f} % d'erreur d'identification</b>".format(kmin[0],erreurs[0]*100),
    placeholder=' ',
    description=' ',
)

file = []
file2 = []
image = []
image2 = []
for i in range(4):
    file.append(open("Erreur{}.png".format(i), "rb"))
    file2.append(open("Mispredicted{}.png".format(i), "rb"))
    image.append(file[-1].read())
    image2.append(file2[-1].read())
erreur = Image(
    value=image[0],
    format='png',
    width=600,
    height=300,
)

mispredicted = Image(
    value=image2[0],
    format='png',
    width=600,
    height=300,
)

display(train)
display(test)

def on_value_change(change):
    train.value = r"Taille des données d'entrainement: {} éléments".format(int(valeurs[indice.value][0])*8//10)
    test.value = r"Taille des données de test: {} éléments".format(int(valeurs[indice.value][0])*2//10)
    erreur.value = image[indice.value]
    mispredicted.value = image2[indice.value]
    titreerreur.value="<b>Erreur pour différentes valeurs de k avec un échantillon de {} éléments</b>".format(int(valeurs[indice.value][0]))
    valeurk.value = r"<b>La meilleure valeur de k est {} correspondant à {:.2f} % d'erreur d'identification</b>".format(kmin[indice.value],erreurs[indice.value]*100)
    
indice.observe(on_value_change, names='value')

Dropdown(description='Eléments:', options=(('6000', 0), ('3000', 1), ('2000', 2), ('1500', 3)), value=0)

HTMLMath(value="Taille des données d'entrainement: 4800 éléments", description=' ', placeholder=' ')

HTMLMath(value='Taille des données de test: 1200 éléments', description=' ', placeholder=' ')

```python
Nbre_elements = 6000 #ou 3000 ou 2000 ou 1500
sample = np.random.randint(len(X), size=Nbre_elements)
xsample=X[sample]
ysample=y[sample]
xtrain, xtest, ytrain, ytest = train_test_split(xsample, ysample, train_size=0.8)
```

### Mesure de l'erreur réalisée en fonction du nombre k de voisins considérés

Calcul du pourcentage d'erreur d'identification en fonction du k choisi (on doit prendre le k pour lequel l'erreur est la plus faible)

```python
fig3=plt.figure("Figure 3")
ax3=fig3.add_subplot(1,1,1)
errors = []
for k in range(2,15):
    knn = neighbors.KNeighborsClassifier(k)
    errors.append(100*(1 - knn.fit(xtrain, ytrain).score(xtest, ytest)))
ax3.plot(range(2,15), errors, 'o-')
ax3.set_xlabel("valeur de k")
ax3.set_ylabel("pourcentage d'erreur")
fig3.show()
```

In [4]:
display(titreerreur)
display(erreur)

HTMLMath(value='<b>Erreur pour différentes valeurs de k avec un échantillon de 6000 éléments</b>', description…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x85\x00\x00\x01\n\x08\x06\x00\x00\x00\xfd\xc5IE\…

### On ajuste le modèle par rapport aux données d'entrainement en considérant le meilleur k, puis on utilise le modèle obtenu sur les données de test pour essayer de les identifier. 

In [5]:
display(valeurk)

HTMLMath(value="<b>La meilleure valeur de k est 6 correspondant à 5.67 % d'erreur d'identification</b>", descr…

```python
knn = neighbors.KNeighborsClassifier(2+errors.index(min(errors)))
knn.fit(xtrain, ytrain)

# On récupère les prédictions sur les données test
predicted = knn.predict(xtest)
```

### Voici des exemples d'images mal identifiées

In [10]:
display(mispredicted)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01l\x00\x00\x01\x08\x08\x06\x00\x00\x00{\xfa\xce\xf…